In [1]:
import pandas as pd
import json
from openai import AzureOpenAI

In [11]:
client = AzureOpenAI(
  api_key = GPT4_KEY,  
  api_version = "2024-05-01-preview",
  azure_endpoint = GPT4_ENDPOINT
)

In [ ]:
df = pd.read_excel("KPI Names Shortening - 7.24.xlsx", engine="calamine", usecols="B")
df["Topics"] = ""
df

In [30]:
res = {}

for i in range(0, len(df), 9):
    try:
        data = df.iloc[i:i+9].set_index("KPI Name").squeeze().to_dict()
        data = json.dumps(data, ensure_ascii=False)
        response = client.chat.completions.create(
            model="gpt-4o",
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": """
                                            You are an AI assitant that generate topics based on descriptions.
                                            Do not use words different from the descriptions to generate the topics.
                                            Keep the topics short.
                                            You will be provided a JSON object where the keys are the descriptions and you must fill in the corresponding values with the generated topics.
                                            Only return the JSON object with the filled in topics in the response.

                                            For example:
                                            {"Disclose a confirmation that the scope of consolidation is the same as for the financial statements, or, where applicable, a declaration that the reporting undertaking is not required to draw-up financial statements or that the reporting undertaking is preparing consolidated sustainability reporting pursuant to Article 48i of Directive 2013/34/EU":"Scope of Consolidation and Reporting Requirements"}"""},
                {"role": "user", "content": "Generate topics for the following:{0}".format(data)}
            ]
        )
        
        if response.choices[0].finish_reason != "stop":
            print(i)
            print(response.model_dump_json(indent=2))
            break

        res.update(json.loads(response.choices[0].message.content))

    except Exception as e:
        print(f"Error: {e}")
        print(i)
        break


In [ ]:
result = pd.DataFrame(list(res.items()), columns=['Description', 'Topic'])
result

In [ ]:
excel_file = '1371.xlsx'
result.to_excel(excel_file, index=False)
print(f"Excel file '{excel_file}' has been created.")